# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [10]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The CSV file titled <font color=red>event_datafile_new.csv</font> is located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION= {'class':'SimpleStrategy', 
    'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## The tables I created are suited to the following three queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In all tables, strictly the necessary columns are included. For example in this query session_id and item_in_session are needed for the primary key and the rest of the columns are needed for return the appropriate data of the query.
## Task 1: Getting the artist, song title and length for a given session id and item in session
For the first query, I chose session_id as a partition key and item_in_session as a clustering column, because that was the order in which the query was given. Also, a session can have one or more items in it, so it makes sense that session_id will have the role of a key. 

In [8]:
query = "CREATE TABLE IF NOT EXISTS artist_listened (session_id int, item_in_session int,\
        artist text, song text, length float, \
        PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
query1 = "SELECT artist, song, length FROM artist_listened WHERE \
        session_id=338 AND item_in_session=4"                   

In [9]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_listened (session_id, item_in_session, \
        artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
table = PrettyTable()
table.field_names = ["Artist", "Song", "Length"]
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
for row in rows:
    table.add_row([row.artist, row.song, row.length])
print(table)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


## Task 2: Getting the artist, his/her song and the user's name for a given user and session id. Sort by item in session
I chose to have a composite partition key to ensure there will be no duplicates of this key. It consists of an user and session id. Finally, I included the item_in_session as a clustering column because that way the results will be sorted by it.

In [12]:
query = "CREATE TABLE IF NOT EXISTS user_sessions (user_id int, session_id int, item_in_session int, \
    artist text, song text, first_name text, last_name text,  \
    PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_sessions (user_id, session_id, item_in_session, \
        artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))                   

In [14]:
query2 = "SELECT artist, song, first_name, last_name FROM user_sessions WHERE user_id=10 AND session_id=182"
table = PrettyTable()
table.field_names = ["Artist", "Song", "First name", "Last name"]
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
for row in rows:
    table.add_row([row.artist, row.song, row.first_name, row.last_name])
print(table)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First name | Last name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


## Task 3: Getting the user names for a given song.
I chose the song as a partiton key because most likely there won't be two songs with the same name. The user_id is a clustering column because we need to query on it. 

In [23]:
query = "CREATE TABLE IF NOT EXISTS user_listened (song text, user_id int, \
    first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listened (song, user_id, \
        first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))                 

In [25]:
query3 = "SELECT first_name, last_name FROM user_listened WHERE song='All Hands Against His Own'"
table = PrettyTable()
table.field_names = ["First name", "Last name"]
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
for row in rows:
    table.add_row([row.first_name, row.last_name])
print(table)

+------------+-----------+
| First name | Last name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Dropping the tables before closing out the sessions

In [26]:
drop_queries = []
drop_queries.append("DROP TABLE IF EXISTS artist_listened")
drop_queries.append("DROP TABLE IF EXISTS user_listened")
drop_queries.append("DROP TABLE IF EXISTS user_sessions")
for query in drop_queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()